<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>4. Usar modelos Keras con Scikit-learn</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Introducción](#section1)
* [2. Evaluar modelos con validación cruzada](#section2)
* [3. Optimización de hiperparámetros](#section3)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En esta lección, aprenderemos cómo usar modelos de Deep Learning de Keras con Scikit-learn en Python. Después de completar esta lección, sabrá:
* Cómo ajustar un modelo de Keras para usarlo con Scikit-learn.
* Cómo evaluar fácilmente los modelos de Keras mediante la validación cruzada en scikit-learn.
* Cómo ajustar los hiperparámetros del modelo de Keras mediante la búsqueda GridSearchCV en scikit-learn.

In [ ]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Introducción</font>

Scikit-Learn es una librería con todas las funciones de uso general y proporciona muchas utilidades que son útiles en el desarrollo de modelos de Deep Learning. No menos importante:
* Evaluación de modelos utilizando métodos de remuestreo como la validación cruzada de k-fold.
* Búsqueda y evaluación eficiente de hiperparámetros del modelo.

La biblioteca de Keras proporciona un contenedor _(wrapper)_ conveniente para que los modelos de Deep Learning se utilicen como estimadores de clasificación o regresión en scikit-learn. En las siguientes secciones, trabajaremos a través de ejemplos del uso del contenedor `KerasClassifier`. 


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Evaluar modelos con validación cruzada </font>

Las clases `KerasClassifier` y `KerasRegressor` en Keras toman un argumento `build_fn` que es el nombre de la función a llamar para crear su modelo. Debes definir una función llamada lo que quieras que defina tu modelo, lo compile y lo devuelva. En el siguiente ejemplo, 

1. Definimos una función `create_model()` que crea una red neuronal MLP.
2. Pasamos este nombre de función a la clase `KerasClassifier` mediante el argumento `build_fn`.
3. También pasamos argumentos adicionales de `epochs = 150` y `batch_size = 10`. Estos se agrupan automáticamente y se pasan a la función `fit()` que es llamada internamente por la clase `KerasClassifier`. 
4. Utilizamos la clase `StratifiedKFold` de Scikit-Learn con 10-fold. 
5. Usamos la función scikit-learn `cross_val_score()` para evaluar nuestro modelo.

In [9]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy as np

# Function to create model, required for KerasClassifier
def create_model():
  model = Sequential()
  model.add(Dense(12, input_dim=8, activation = 'relu'))
  model.add(Dense(8, activation = 'relu'))
  model.add(Dense(1, activation = 'sigmoid'))
  # compile
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

# load pima indians dataset
dataset = np.loadtxt('https://raw.githubusercontent.com/FMunyoz/colab/main/Datasets/pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
y = dataset[:, 8]

# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits=10, shuffle=True)
resulsts = cross_val_score(model, X, y, cv=kfold)
print(resulsts.mean())

0.6926691710948945


Puede ver que cuando se ajusta el modelo de Keras, la estimación de la precisión del modelo puede simplificarse en gran medida, en comparación con la enumeración manual de validación cruzada realizada en la lección anterior.

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Optimización de hiperparámetros</font>

En este ejemplo, usamos `GridSearchCV` para evaluar diferentes configuraciones para nuestro modelo de red neuronal e informar sobre la combinación que proporciona el mejor rendimiento estimado. 

La función `create_model()` está definida para tomar dos argumentos `optimizer` e `init`, los cuales deben tener valores predeterminados. Específicamente, los hiperparámetros a establecer serán:
* Optimizadores para establecer los pesos.
* Inicializadores para los pesos.
* Número de épocas para entrenar el modelo.
* Batchs para variar el número de muestras antes de las actualizaciones de peso.

Las opciones se especifican en un diccionario y se pasan a la configuración de la clase `GridSearchCV`.

In [2]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
import numpy as np

# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop',init='glorot_uniform'):
  model = Sequential()
  model.add(Dense(12, input_dim=8, kernel_initializer=init, activation = 'relu'))
  model.add(Dense(8, kernel_initializer=init, activation = 'relu'))
  model.add(Dense(1, kernel_initializer=init, activation = 'sigmoid'))
  # compile
  model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model

# load pima indians dataset
dataset = np.loadtxt('https://raw.githubusercontent.com/FMunyoz/colab/main/Datasets/pima-indians-diabetes.csv', delimiter=',')
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
y = dataset[:, 8]

# create model
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)

# grid search epochs, batch size and optimizer

optimizers = ['rmsprop', 'adam']
inits = ['glorot_uniform','normal', 'uniform']
epochs = [50,100,150]
batches = [5, 10, 20]

param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
grid = GridSearchCV(estimator=model, param_grid= param_grid, cv=3)
grid_result = grid.fit(X, y)
# summarize results
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) con: %r" % (mean, stdev, param))

Mejor: 0.730469 usando {'batch_size': 10, 'epochs': 150, 'init': 'normal', 'optimizer': 'rmsprop'}
0.546875 (0.119636) con: {'batch_size': 5, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.658854 (0.025780) con: {'batch_size': 5, 'epochs': 50, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.688802 (0.035277) con: {'batch_size': 5, 'epochs': 50, 'init': 'normal', 'optimizer': 'rmsprop'}
0.723958 (0.030978) con: {'batch_size': 5, 'epochs': 50, 'init': 'normal', 'optimizer': 'adam'}
0.712240 (0.033197) con: {'batch_size': 5, 'epochs': 50, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.699219 (0.014616) con: {'batch_size': 5, 'epochs': 50, 'init': 'uniform', 'optimizer': 'adam'}
0.700521 (0.033502) con: {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.688802 (0.007366) con: {'batch_size': 5, 'epochs': 100, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.722656 (0.008438) con: {'batch_size': 5, 'epochs': 100, 'init': 'normal', 'optimize

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>